In [14]:
# move workidir to the correct folder
import os
os.chdir("../")


In [15]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import pytorch_lightning as pl

from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# Custom Libs
from dataset.dataset import Dataset
from model.models.listings_regressor import MLPRegressor, LossAccCallback

In [16]:
# Set up which conlumn to use as a target :
# 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
# 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'price'

REFERENCE_COlUMN = "price"
OTLIERS_FILTERS = 500

In [17]:
# Experiments Reproducibility
from pytorch_lightning import seed_everything

seed_everything(42, workers=True)

Global seed set to 42


42

In [18]:
# Load the precleaned and spearated datset
with open(f'./dataset/train_dataset.pkl', 'rb') as f:
    train_dataset = pd.read_pickle(f)

with open(f'./dataset/test_dataset.pkl', 'rb') as f:
    test_dataset = pd.read_pickle(f)

FileNotFoundError: [Errno 2] No such file or directory: './dataset/train_dataset.pkl'

In [ ]:
y_train = train_dataset.getAllScores()[REFERENCE_COlUMN]
x_not_embeddings_train = train_dataset.getListingsNotEmbeddings()
x_embeddings_train = train_dataset.getListingsEmbeddings()
print("not_embeddings_shape", x_not_embeddings_train.shape)
print("embeddings_shape", x_embeddings_train.shape)

In [ ]:
# Apply PCA to non embeddings fields + normalization
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Note that the PCA and the scaler are kept to be reused in the test
pca = PCA(n_components=60)
scaler = StandardScaler()

x_not_embeddings_train = x_not_embeddings_train.fillna(0)
x_not_embeddings_train_scaled = scaler.fit_transform(x_not_embeddings_train)
x_not_embeddings_train_60 = pca.fit_transform(x_not_embeddings_train_scaled)

In [ ]:
#  define X and y for training
X_train = np.concatenate((x_not_embeddings_train_60, x_embeddings_train), axis=1)   

In [ ]:
# Load and prepare Test
y_test = test_dataset.getAllScores()[REFERENCE_COlUMN]
x_not_embeddings_test = test_dataset.getListingsNotEmbeddings()
x_embeddings_test = test_dataset.getListingsEmbeddings()
print("not_embeddings_shape", x_not_embeddings_test.shape)
print("embeddings_shape", x_embeddings_test.shape)

# PCA for dimensionality reduction
pca = PCA(n_components=60)
scaler = StandardScaler()

x_not_embeddings_test = x_not_embeddings_test.fillna(0)
x_not_embeddings_test_scaled = scaler.fit_transform(x_not_embeddings_test)
x_not_embeddings_test_60 = pca.fit_transform(x_not_embeddings_test_scaled)

X_test = np.concatenate((x_not_embeddings_test_60, x_embeddings_test), axis=1)

In [ ]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

# Drop nan values based on target conlumn
empty_target_indexes_train = y_train.notna()
X_train = X_train[empty_target_indexes_train]
y_train = y_train[empty_target_indexes_train]

empty_target_indexes_test = y_test.notna()
X_test = X_test[empty_target_indexes_test]
y_test = y_test[empty_target_indexes_test]

## Appling cutout
not_outliers_idx = y_train <= OTLIERS_FILTERS
X_train = X_train[not_outliers_idx]
y_train = y_train[not_outliers_idx]

not_outliers_idx = y_test <= OTLIERS_FILTERS
X_test = X_test[not_outliers_idx]
y_test = y_test[not_outliers_idx]

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
metrics_df = pd.DataFrame()
mse = []
rmse = []
r2 = []
mae = []
model_names = []
x_range = range(len(y_test)) # used to plot data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# compute the mean squared error, RMSE and R2 score and MAE and show the results on a dataframe, append the results of each model
# in a list and then create a dataframe with the results
# compute the mean squared error, RMSE and R2 score and MAE and show the results on a dataframe, append the results of each model
# in a list and then create a dataframe with the results
def compute_metrics(model, X_test, y_test):

    if model.__class__.__name__ == 'MLPRegressor':
        y_pred = model(torch.tensor(X_test).float()).detach().numpy()
        y_pred = y_pred.reshape(-1)
        
    else:
        y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    model_name = model.__class__.__name__

    # create a temporary dataframe with the results for this model
    temp_df = pd.DataFrame({
        'model': [model_name],
        'mse': [mse],
        'rmse': [rmse],
        'r2': [r2],
        'mae': [mae]
    })

    # append the temporary dataframe to the metrics_df dataframe
    global metrics_df
    metrics_df = pd.concat([metrics_df, temp_df], ignore_index=True)
    
    print(metrics_df)


### Random forest regressor


In [ ]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# create the model
rf = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42)

# train the model
rf.fit(X_train, y_train)

# predict
y_hat = rf.predict(X_test)

# compute the mean squared error
print(mean_squared_error(y_test, y_hat))

KeyboardInterrupt: 

In [ ]:
compute_metrics(rf, X_test, y_test)

In [ ]:
# create a dataframe with the y and y_hat next to each other
y_test = y_test
df = pd.DataFrame({'y': y_test, 'y_hat': y_hat})
df.sort_values(by=['y']).head(10)

In [ ]:
# plot accuracy
plt.figure(figsize=(15, 8))
plt.plot(x_range[:100], y_test[:100], 'o-', label='y')
plt.plot(x_range[:100], y_hat[:100], 'o-', label='y_hat')
plt.legend()
plt.show()

### let's try a different model, a gradient boosting regressor


In [ ]:

from sklearn.ensemble import GradientBoostingRegressor

# create the model
gb = GradientBoostingRegressor(n_estimators=20, max_depth=10, random_state=42)

# train the model
gb.fit(X_train, y_train)

# predict
y_hat = gb.predict(X_test)

#  compute metrics
compute_metrics(gb, X_test, y_test)

In [ ]:
# create a dataframe with the y and y_hat next to each other
y_hat = y_hat
y_test = y_test
df = pd.DataFrame({'y': y_test, 'y_hat': y_hat})
df.sort_values(by=['y']).head(10)

In [ ]:
# plot accuracy
plt.figure(figsize=(15, 8))
plt.plot(x_range[:100], y_test[:100], 'o-', label='y')
plt.plot(x_range[:100], y_hat[:100], 'o-', label='y_hat')
plt.legend()
plt.show()

### LGBMRegressor

In [ ]:
from lightgbm import LGBMRegressor


# create the model
lgbm = LGBMRegressor(n_estimators=100, max_depth=10, random_state=42)

# train the model
lgbm.fit(X_train, y_train)

# predict
y_hat = lgbm.predict(X_test)

# compute the loss metrics
compute_metrics(lgbm, X_test, y_test)

In [ ]:
# plot the y and y_hat on two different graphs, plot  the first 100 values, plot in dots and lines
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 8))
plt.plot(x_range[:100], y_test[100:200], 'o-', label='y')
plt.plot(x_range[:100], y_hat[100:200], 'o-', label='y_hat')
plt.legend()


In [ ]:
# create a dataframe with the y and y_hat next to each other
y_hat = y_hat
y_test = y_test
df = pd.DataFrame({'y': y_test, 'y_hat': y_hat})
df.sort_values(by=['y']).head(10)

In [ ]:
# Pre-Launch Tensorboard
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

#  Tensorboard webpage : http://localhost:6006

In [ ]:

# split data
X_train_1, X_val, y_train_1, y_val = train_test_split(X_train, y_train.values, test_size=0.2, random_state=42)

# convert data to float32
X_train = X_train.astype(np.float32)
X_train_1 = X_train_1.astype(np.float32)
X_val = X_val.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_train_1 = y_train_1.astype(np.float32)
y_val = y_val.astype(np.float32)
y_test = y_test.astype(np.float32)

# Define the PyTorch DataLoader for training and testing data
train_dataset = TensorDataset(torch.tensor(X_train_1), torch.tensor(y_train_1))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(torch.tensor(X_val), torch.tensor(y_val))
val_loader = DataLoader(val_dataset, batch_size=32)

test_dataset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test.values))
test_loader = DataLoader(val_dataset, batch_size=32)

# Define the MLPRegressor model
model = MLPRegressor()

# Define the PyTorch Lightning Trainer
trainer = pl.Trainer(callbacks=[LossAccCallback()],
                    max_epochs=50, accelerator="auto", devices="auto", strategy="auto")
# trainer = pl.Trainer(accelerator="auto", devices="auto", strategy="auto", max_epochs=10)

# Train the model
trainer.fit(model, train_loader, val_loader)
compute_metrics(model, X_test, y_test)

# Plots 
# create a dataframe with the y and y_hat next to each other
y_hat = model(torch.tensor(X_test).float()).detach().numpy()
y_hat = y_hat.reshape(-1)
y_test = y_test
df = pd.DataFrame({'y': y_test, 'y_hat': y_hat})

import matplotlib.pyplot as plt
plt.figure(figsize=(15, 8))
plt.plot(x_range[:100], df.y[:100], 'o-', label='y')
plt.plot(x_range[:100], df.y_hat[:100], 'o-', label='y_hat')
plt.legend()
plt.show()